# Target investigation


### Packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import functions    # my own functions which are used in more notebooks
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
#import warnings
#import numpy as np
#import tqdm
#import math
#import kds
#import pickle
#from tqdm import tqdm
#from scipy.sparse import csr_matrix
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
#import gensim
#print((gensim.__version__))  
#from gensim.models import Word2Vec
#import gensim.downloader as api
#from gensim.models import KeyedVectors
#warnings.filterwarnings('ignore')
#from sklearn.metrics import roc_curve, auc, roc_auc_score,classification_report, accuracy_score,precision_score,recall_score
#from re import search
#from sklearn.ensemble import RandomForestClassifier

## Upload cleaned abstracts

In [60]:
df_all = pd.read_csv("1.Preprocessing_outputs/df_sw_tok_low_punc_lemm_v6.csv").rename(columns = {'doi_x':'doi'}).set_index("doi")

In [61]:
len(df_all)

483636

In [62]:
df_all.head(2)

,Unnamed: 0,Year,Month,abstract,cord_uid,len,abstract_cleaned
doi,,,,,,,
10.1186/1471-2334-1-6,0,2001.0,7.0,OBJECTIVE: This retrospective chart review des...,ug7v899j,1158,retrospective chart review describes epidemiol...
10.1186/rr14,1,2000.0,8.0,Inflammatory diseases of the respiratory tract...,02tnwd4m,718,inflammatory disease respiratory tract commonl...


# Analýza

In [63]:
df_new = df_all.reset_index().copy()
df_all = df_all.reset_index()

for year_cit in list(range(2015,2023)):
    print(year_cit)
    df_new = functions.add_target_opencitatins_marginal(target_year = year_cit,df = df_all,target_col_name = str(year_cit))
    df_new[str(year_cit)] = df_new[str(year_cit)]
    df_all = df_all.merge(df_new[["doi",str(year_cit)]],on="doi",how="left")
    df_all[str(year_cit)] = df_all[str(year_cit)].fillna("empty")

2015


C:\Users\berl03\AppData\Local\Temp/ipykernel_13980/1630723808.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  df_new = functions.add_target_opencitatins_marginal(target_year = year_cit,df = df_all,target_col_name = str(year_cit))
b'Skipping line 171761: expected 5 fields, saw 8\nSkipping line 208878: expected 5 fields, saw 8\n'


0    5711
1    4166
Name: 2015, dtype: int64
9877
2016


b'Skipping line 171761: expected 5 fields, saw 8\nSkipping line 208878: expected 5 fields, saw 8\n'


0    6439
1    4958
Name: 2016, dtype: int64
11397
2017


...

0    22366
1    17799
Name: 2020, dtype: int64
40165
2021


b'Skipping line 171761: expected 5 fields, saw 8\nSkipping line 208878: expected 5 fields, saw 8\n'


0    8588
1    5002
Name: 2021, dtype: int64
13590
2022


b'Skipping line 171761: expected 5 fields, saw 8\nSkipping line 208878: expected 5 fields, saw 8\n'


0    20
1     1
Name: 2022, dtype: int64
21


In [64]:
df_plot = df_all[["Year"] + list(map(str,list(range(2015,2023))))].set_index("Year").stack().reset_index().rename({0:"target"},axis=1).value_counts().reset_index().rename({0:"cnt"},axis=1)

min_publ_year = 2015
min_cit_year  = 2017

df_plot = df_plot.sort_values("Year",ascending = False)
df_plot_f = df_plot[df_plot["Year"].astype(int)>=min_publ_year]
df_plot_f = df_plot_f[df_plot_f["level_1"].astype(float).astype(int)>=min_cit_year]
df_piv = df_plot_f.pivot(index="Year", columns = ["level_1","target"],values="cnt").fillna(0).round(0).astype(int).sort_index(axis=1)
df_piv["Total_cnt_public_arcls"] = df_piv[('2017',0.0)]+df_piv[('2017',1.0)]+df_piv[('2017',"empty")]
del df_piv[('2017',"empty")]
df_piv.loc['Total'] = df_piv.sum(numeric_only=True)
df_piv

In [73]:
df_piv

level_1  2017        2018                2019                 2020         \
target    0.0   1.0   0.0   1.0   empty   0.0   1.0   empty    0.0    1.0   
Year                                                                        
2015.0    887   742   876   735    4623   840   767    4627    901    774   
2016.0    962   629   903   799    4584   867   861    4558    918    871   
2017.0    375   368   979   616    4917   894   874    4744   1057    797   
2018.0     13     0   403   397    5812  1020   771    4821   1040    997   
2019.0      2     1    13     1    7975   532   497    6960   1284   1248   
2020.0      9     0     5     0  126335   120     6  126214  11955   8727   
2021.0      0     0     0     0  229139     0     0  229139     53     29   
2022.0      0     0     0     0   56884     0     0   56884      0      0   
2023.0      0     0     0     0       1     0     0       1      0      0   
Total    2248  1740  3179  2548  440270  4273  3776  437948  17208  13443   

level_1          2021               2022             Total_cnt_public_arcls  
target    empty   0.0   1.0   empty  0.0 1.0   empty                         
Year                                                                         
2015.0     4559   289   109    5836    1   0    6233                   6234  
2016.0     4497   305   157    5824    2   0    6284                   6286  
2017.0     4658   341   139    6032    1   1    6510                   6512  
2018.0     4575   350   167    6095    1   0    6611                   6612  
2019.0     5457   441   192    7356    0   0    7989                   7989  
2020.0   105658  5204  3458  117678   11   0  126329                 126340  
2021.0   229057    33    10  229096    0   0  229139                 229139  
2022.0    56884     0     0   56884    0   0   56884                  56884  
2023.0        1     0     0       1    0   0       1                      1  
Total    415346  6963  4232  434802   16   1  445980                 445997